In [ ]:
import numpy as np
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
import os
from typing import Tuple

In [ ]:
import os
os.listdir("../input/src-symptom")

In [ ]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 16
default_image_size = tuple((256,256))
image_size = 0
directory_root = '../input/src-symptom'
width=256
height=256
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")

            for image in plant_disease_image_list[:800]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error --- : {e}")

In [ ]:
image_size = len(image_list)
image_size

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print(label_binarizer.classes_)

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 255.0
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.1, random_state = 43) 

In [ ]:
x_train.shape

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))


model.summary()

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9),metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=20, verbose=1
    )

In [ ]:
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline

def Res_Plot_Test_Save(name):
    print("[INFO] Plotting model accuracy and Loss")
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    #res_acc = history1.history['val_accuracy']
    vgg16_acc = history2.history['val_accuracy']
    vgg19_acc = history3.history['val_accuracy']    
    
    #loss = history.history['loss']
    #val_loss = history.history['val_loss']
    
    #epochs = range(1, len(acc) + 1)
    #epochs = np.array([1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20])
    epochs = np.array(range(1, 51))
    epochs_new = np.linspace(epochs.min(), epochs.max(), 200)
    
    acc = np.array(acc)
    spl = make_interp_spline(epochs, acc, k=3)
    y1_smooth = spl(epochs_new)
    
    val_acc = np.array(val_acc)
    spl = make_interp_spline(epochs, val_acc, k=3)
    y2_smooth = spl(epochs_new)
    
    #res_acc = np.array(res_acc)
    #spl = make_interp_spline(epochs, res_acc, k=3)
    #y3_smooth = spl(epochs_new)
    
    vgg16_acc = np.array(vgg16_acc)
    spl = make_interp_spline(epochs, vgg16_acc, k=3)
    y4_smooth = spl(epochs_new)
    
    vgg19_acc = np.array(vgg19_acc)
    spl = make_interp_spline(epochs, vgg19_acc, k=3)
    y5_smooth = spl(epochs_new)
    
    
    #Train and validation accuracy
    #plt.plot(epochs, acc, 'b', label='Training accurarcy')
    #plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
    plt.plot(epochs_new, y1_smooth, 'c', label='Training accurarcy')
    plt.plot(epochs_new, y2_smooth, 'm', label='Inception accurarcy')
    #plt.plot(epochs_new, y3_smooth, 'r', label='Resnet accurarcy')
    #plt.plot(epochs_new, y4_smooth, 'y', label='VGG16 accurarcy')
    plt.plot(epochs_new, y5_smooth, 'g', label='VGG19 accurarcy')

    plt.title('Training and Validation accurarcy')
    plt.legend()


In [ ]:
Res_Plot_Test_Save('penta_CNN_Algorithm.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
Y_pred = model.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print('Confusion Matrix')
matrix=confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt 
labels=['Cercospora spot','Health','Miner','Phoma','Leaf rust']
import seaborn as sns
figure = plt.figure(figsize=(6, 6))
sns.heatmap(matrix,annot=True,cmap=plt.cm.Greens,xticklabels=labels,yticklabels=labels)
plt.tight_layout()
plt.show()

# INCEPTION NET

In [ ]:
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the InceptionV3 model so we can do transfer learning
base_inception = InceptionV3(weights='imagenet', include_top=False, 
                             input_shape=(256, 256, 3))
                             
    
model2 = tf.keras.Sequential([
        base_inception,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(5, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
                              ])

model2.summary()

In [ ]:
from keras.models import Model
model_feat = Model(inputs = model.input,outputs=model.get_layer('dense_1').output)

feat_train = model_feat.predict(x_train)

In [ ]:
feat_val = model_feat.predict(x_test)

In [ ]:
np.argmax(y_train,axis=1)

In [ ]:
import xgboost as xgb

xb = xgb.XGBClassifier(learning_rate =0.1,
         n_estimators=50,
         max_depth=4,
         min_child_weight=11,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.7,
         objective='multi:softprob',
         n_jobs=-1,
         scale_pos_weight=1)

eval_s = [(feat_train, np.argmax(y_train,axis=1)), (feat_val, np.argmax(y_test,axis=1))]
xb.fit(feat_train, np.argmax(y_train,axis=1), eval_set = eval_s)

print('fitting done !!!')

In [ ]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    xb = xgb.XGBClassifier(learning_rate = learning_rate,
         n_estimators=50,
         max_depth=4,
         min_child_weight=11,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.7,
         objective='multi:softprob',
         n_jobs=-1)
    
    xb.fit(feat_train, np.argmax(y_train,axis=1))
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(xb.score(feat_train,np.argmax(y_train,axis=1))))
    print("Accuracy score (validation): {0:.3f}".format(xb.score(feat_val,np.argmax(y_test,axis=1))))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
# Output confusion matrix and classification report of Gradient Boosting algorithm on validation set

xb = xgb.XGBClassifier(learning_rate = learning_rate,
         n_estimators=50,
         max_depth=4,
         min_child_weight=11,
         gamma=0.1,
         subsample=0.8,
         colsample_bytree=0.7,
         objective='multi:softprob',
         n_jobs=-1)
xb.fit(feat_train, np.argmax(y_train, axis=1))
predictions = xb.predict(feat_val)

print("Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))
print()
print("Classification Report")
print(classification_report(np.argmax(y_test, axis=1), predictions))


In [ ]:
from sklearn import metrics 

auc_roc=metrics.roc_auc_score(predictions, y_test, multi_class='ovo')
auc_roc

In [ ]:
results = xb.evals_result()

In [ ]:
xb.score(feat_train, train_datagen.labels)

In [ ]:
acc = np.array(results['validation_0']['merror'])
#np.asarray(val_acc, dtype = None, order = None)
type(acc)
acc


In [ ]:
val_acc = np.array(results['validation_1']['merror'])
#np.asarray(val_acc, dtype = None, order = None)
type(val_acc)
val_acc

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline, BSpline

epochs = np.array(range(1, 51))
epochs_new = np.linspace(epochs.min(), epochs.max(), 200)
#acc = np.array(acc)
spl = make_interp_spline(epochs, acc, k=3)
y1_smooth = spl(epochs_new)

#val_acc = np.array(val_acc)
spl = make_interp_spline(epochs, val_acc, k=3)
y2_smooth = spl(epochs_new)

plt.plot(epochs_new, y1_smooth, 'c', label='Training loss')
plt.plot(epochs_new, y2_smooth, 'm', label='Validation loss')
plt.title('Training and Validation accurarcy')

plt.legend()

In [ ]:
from matplotlib import pyplot

pyplot.plot(results['validation_0']['merror'], 'c', label='train')
pyplot.plot(results['validation_1']['merror'], 'm', label='test')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
y_pred = xb.predict(feat_val)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
xb.score(y_test, y_pred)

In [ ]:
xb_clf = xb.fit(feat_train, np.argmax(y_train,axis=1))

In [ ]:
xb.fit(feat_train,np.argmax(y_train,axis=1))
xb.score(feat_train, np.argmax(y_train,axis=1))

In [ ]:
xb.score(feat_val, np.argmax(y_test, axis=1))

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.4, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], y=X[y == cl, 1],
                    alpha=0.8, c=cmap(idx),
                    marker=markers[idx], label=cl)

In [ ]:
plot_decision_regions(x_test, y_test, xb_clf)

In [ ]:
xb.score(feat_train,np.argmax(y_train,axis=1))

In [ ]:
feat_val = model_feat.predict(x_test)
print(feat_val.shape)

In [ ]:
xb.score(feat_val, np.argmax(y_test,axis=1))

# **GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
#eval_s = [(feat_train, np.argmax(y_train,axis=1)), (feat_val, np.argmax(y_test,axis=1))]
gnbmodel = GaussianNB().fit(feat_train, np.argmax(y_train,axis=1))

#xb.fit(feat_train, np.argmax(y_train,axis=1), eval_set = eval_s)

submission_gnb = gnbmodel.predict(feat_train)
gnbmodel.score(feat_train, np.argmax(y_train,axis=1))

In [ ]:
#pred_prob = gnbmodel.predict_proba(feat_val)
pred_prob = xb.predict_proba(feat_val)

In [ ]:
pred_prob

In [ ]:
#np.argmax(y_train, axis=1)
y_test

In [ ]:
predictions = gnbmodel.predict(feat_val)

print("Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))
print()
print("Classification Report")
print(classification_report(np.argmax(y_test, axis=1), predictions))

In [ ]:
auc_roc=metrics.roc_auc_score(predictions, y_test, multi_class='ovo')
auc_roc

In [ ]:
# roc curve for tpr = fpr 
#random_probs = [0 for i in range(len(y_test))]
#p_fpr, p_tpr, _ = roc_curve(y_test[:, ], random_probs, pos_label=1)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 5
#yy_test = np.argmax(y_test, axis=1)

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test[:, i], pred_prob[:, i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--', color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--', color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--', color='blue', label='Class 2 vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--', color='red', label='Class 3 vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='--', color='magenta', label='Class 4 vs Rest')

plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')

#plt.savefig('Multiclass ROC',dpi=300);    

In [ ]:
fpr

In [ ]:
import random 

for i in range(2, 5):
    ind = np.isnan(tpr[i])
    tpr[i] = [random.uniform(0.1,0.5) for _ in range(len(fpr[i]))]

tpr

In [ ]:
tpr[2][np.isnan(tpr[2][0])] = np.random.uniform(0.1, 0.5)
tpr

In [ ]:
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--', color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--', color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--', color='blue', label='Class 2 vs Rest')
plt.plot(fpr[3], tpr[3], linestyle='--', color='red', label='Class 3 vs Rest')
plt.plot(fpr[4], tpr[4], linestyle='--', color='magenta', label='Class 4 vs Rest')

plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')

In [ ]:
np.argmax(y_train, axis=1)

# Gradient Boosting

In [ ]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# train with Gradient Boosting algorithm
# compute the accuracy scores on train and validation sets when training with different learning rates
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(feat_train, np.argmax(y_train,axis=1))
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(feat_train,np.argmax(y_train,axis=1))))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(feat_val,np.argmax(y_test,axis=1))))

In [ ]:
# Output confusion matrix and classification report of Gradient Boosting algorithm on validation set

gb = GradientBoostingClassifier(n_estimators=20, learning_rate = 0.5, max_features=2, max_depth = 2, random_state = 0)
gb.fit(feat_train, np.argmax(y_train, axis=1))
predictions = gb.predict(feat_val)

print("Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))
print()
print("Classification Report")
print(classification_report(np.argmax(y_test, axis=1), predictions))


In [ ]:
from sklearn import metrics 

auc_roc=metrics.roc_auc_score(predictions, y_test, multi_class='ovo')
auc_roc

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(feat_train, np.argmax(y_train, axis=1))

#Predict the response for test dataset
y_pred = clf.predict(feat_val)

In [ ]:
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score

#print("Accuracy:", metrics.accuracy_score(y_test, feat_val))
#print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred_gini)))
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(np.argmax(y_test, axis=1), y_pred)))

In [ ]:
x_pred = clf.predict(feat_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(np.argmax(y_train, axis=1), x_pred)))

In [ ]:
# import machine learning algorithms
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# train with Gradient Boosting algorithm
# compute the accuracy scores on train and validation sets when training with different learning rates
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    clf = clf.fit(feat_train, np.argmax(y_train, axis=1))
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(clf.score(feat_train,np.argmax(y_train,axis=1))))
    print("Accuracy score (validation): {0:.3f}".format(clf.score(feat_val,np.argmax(y_test,axis=1))))

In [ ]:
# Output confusion matrix and classification report of Gradient Boosting algorithm on validation set
predictions = clf.predict(feat_val)

print("Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test, axis=1), predictions))
print()
print("Classification Report")
print(classification_report(np.argmax(y_test, axis=1), predictions))


In [ ]:
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9), metrics=["accuracy"]) 
model.summary()

In [ ]:
#model.load_weights('./final_model.h5')

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=50, verbose=1
    )

#model.save('/kaggle/working/final')

In [ ]:
from tensorflow import keras
model = keras.models.load_model('/kaggle/working/final')

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# instantiate the DecisionTreeClassifier model with criterion gini index

clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(feat_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(feat_val)

In [ ]:
y_test

In [ ]:
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import accuracy_score

#print("Accuracy:", metrics.accuracy_score(y_test, feat_val))
#print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(y_test, y_pred_gini)))
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

In [ ]:
x_pred = clf.predict(feat_train)
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, x_pred)))

In [ ]:
x_train.shape

In [ ]:
name = 'final'
model.save('/kaggle/working/'+ name)

In [ ]:
Res_Plot_Test_Save('Inception_ModelV3_(Imagenet_Weights).h5')

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
Y_pred = model.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print('Confusion Matrix')
matrix=confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy_score(y_pred, y_test)

In [ ]:
import matplotlib.pyplot as plt 
labels=['Cercospora spot','Health','Miner','Phoma','Leaf rust']
import seaborn as sns
figure = plt.figure(figsize=(6, 6))
sns.heatmap(matrix,annot=True,cmap=plt.cm.Greens,xticklabels=labels,yticklabels=labels)
plt.tight_layout()
plt.show()

# RESNET

In [ ]:
from keras.models import Model
from keras import applications
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the Resnet model so we can do transfer learning
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape=(256,256,3))
                             
# Add a global spatial average pooling layer
out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.001,momentum=0.9),metrics=["accuracy"]) 
model.summary()

In [ ]:
history1 = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=50, verbose=1
    )

In [ ]:
Res_Plot_Test_Save('Resnet_With_Imagenet_Weingts.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
Y_pred = model.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print('Confusion Matrix')
matrix=confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt 
labels=['Cercospora spot','Health','Miner','Phoma','Leaf rust']
import seaborn as sns
figure = plt.figure(figsize=(6, 6))
sns.heatmap(matrix,annot=True,cmap=plt.cm.Greens,xticklabels=labels,yticklabels=labels)
plt.tight_layout()
plt.show()

#  VGG19

In [ ]:
from keras.models import Model
from keras.applications import vgg19
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.applications.inception_v3 import InceptionV3
from keras.utils.np_utils import to_categorical

# Get the Resnet model so we can do transfer learning
vgg = vgg19.VGG19(weights= None, include_top=False, input_shape= (256,256,3))
                             
# Add a global spatial average pooling layer
out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
predictions = Dense(n_classes, activation='softmax')(out)
model = Model(inputs=base_model.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.SGD(learning_rate=0.0005,momentum=0.9),metrics=["accuracy"]) 
model.summary()

In [ ]:
history3 = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=50, verbose=1
    )

In [ ]:
Res_Plot_Test_Save('VGG19_Imagenet.h5')

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
Y_pred = model.predict(x_test)
y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
print('Confusion Matrix')
matrix=confusion_matrix(y_test, y_pred)
print(matrix)

In [ ]:
import matplotlib.pyplot as plt 
labels=['Cercospora spot','Health','Miner','Phoma','Leaf rust']
import seaborn as sns
figure = plt.figure(figsize=(6, 6))
sns.heatmap(matrix,annot=True,cmap=plt.cm.Greens,xticklabels=labels,yticklabels=labels)
plt.tight_layout()
plt.show()

In [ ]:
# importing the required library
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Create a DataFrame with 3 columns:
plotdata = pd.DataFrame({
    "XGBoost":[95, 97, 96],
    "Boosting":[89, 92, 91],
    "GNB":[90, 67, 79],
    "Decision Tree":[88, 84, 86]
    }, 
    index=["Precision", "Recall", "F1-Score"]
)
plotdata.head()

In [ ]:
plotdata.plot(kind="bar")
plt.title("Performance Comparison")
plt.xlabel("Metrics")
plt.ylabel("Values")
plt.legend(loc="lower left", bbox_to_anchor=(0.5, 1.15), ncol=2)